In [2]:
#all the imported packages./modules./etc.
import glob
import os
import cv2
import numpy as np

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 40.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
#os.chdir("C://Users/abisn/OneDrive/Desktop/OpenCV/")
#os.getcwd()

'C:\\Users\\abisn\\OneDrive\\Desktop\\OpenCV'

In [16]:
!pip install roboflow

  Using cached roboflow-1.1.7-py3-none-any.whl (58 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached supervision-0.16.0-py3-none-any.whl (72 kB)
  Using cached scipy-1.11.3-cp39-cp39-win_amd64.whl (44.3 MB)
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1


You should consider upgrading via the 'c:\users\abisn\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="OTb9k6B8IEQqdAk5GpWC")
project = rf.workspace("Data-ge8af").project("Data-x0cje")
dataset = project.version(3).download("yolov7")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ForensicSight-3 in yolov7pytorch:: 100%|██████████| 990/990 [00:02<00:00, 461.94it/s] 


In [4]:
#Setup Directories
if not os.path.exists("./Augmentations"):
    os.mkdir("./Augmentations")
directories = ["ReflectionHorizontal", "ReflectionVertical"]
for d in directories:
    if not os.path.exists("./Augmentations/{}".format(d)):
        os.mkdir("./Augmentations/{}".format(d))
    if not os.path.exists("./Augmentations/{}/images".format(d)):
        os.mkdir("./Augmentations/{}/images".format(d))
    if not os.path.exists("./Augmentations/{}/labels".format(d)):
        os.mkdir("./Augmentations/{}/labels".format(d))
if not os.path.exists("./BoundingBoxed"):
    os.mkdir("./BoundingBoxed")
if not os.path.exists("./Augmentations/Rotation"):
    os.mkdir("./Augmentations/Rotation")
for d in ["90", "180", "270"]:
    if not os.path.exists("./Augmentations/Rotation/{}".format(d)):
        os.mkdir("./Augmentations/Rotation/{}".format(d))
    if not os.path.exists("./Augmentations/Rotation/{}/images".format(d)):
        os.mkdir("./Augmentations/Rotation/{}/images".format(d))
    if not os.path.exists("./Augmentations/Rotation/{}/labels".format(d)):
        os.mkdir("./Augmentations/Rotation/{}/labels".format(d))
inputFolder = "./Data/train/images"

In [5]:
def write_file(arr, path, name, aug_type, sample_type):
    if len(arr) > 4:
        wString = "{} {} {} {} {} {} {} {} {}".format(sample_type, arr[1], arr[2], arr[3], arr[4], arr[5], arr[6], arr[7], arr[8])
    else:
        wString = "{} {} {} {} {} {} {} {} {}".format(sample_type, arr[0][0], arr[0][1], arr[1][0], arr[1][1], arr[2][0], arr[2][1], arr[3][0], arr[3][1])
    f = open("{}/{}_{}.txt".format(path, name, aug_type), "a")
    f.write("{}\n".format(wString))

In [6]:
def find_file(n):
    for fname in os.listdir('./Data/train/labels/'):
        if os.path.isfile("./Data/train/labels/{}".format(fname)):
            if n in fname:
                return fname


In [7]:
def process_points(line, width, height):
    chunks = line.split(' ')
    #print(chunks)
    #cv2.circle(image, ((int)((float(chunks[1]) * height)), (int)((float(chunks[2]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[3]) * height)), (int)((float(chunks[4]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[5]) * height)), (int)((float(chunks[6]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[7]) * height)), (int)((float(chunks[8]) * width))), 10, (0,0,255), 10)
    pts = np.array([[(int)((float(chunks[1]) * width)),(int)((float(chunks[2]) * height))],[(int)((float(chunks[3]) * width)), (int)((float(chunks[4]) * height))],[(int)((float(chunks[5]) * width)), (int)((float(chunks[6]) * height))],[(int)((float(chunks[7]) * width)), (int)((float(chunks[8]) * height))], [(int)(chunks[0]),-1]], np.int32)
    return pts

In [8]:
import math

def rotate(origin, points, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    ar  = np.array([], np.float32)
    for point in points:
        px, py = point
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        ar = np.append(ar, [qx,qy])
    return np.array([[ar[0], ar[1]], [ar[2], ar[3]], [ar[4], ar[5]], [ar[6], ar[7]]], np.int32)

In [9]:
#Horizontal Reflection Method Labels
def RH_Coords(chunks):

    relfectedChunks = chunks.copy()

    for chunk in range(len(chunks)):
        if(chunk%2 != 0 and chunk != 0):
            reflectedX = 1 - float(chunks[chunk])
            relfectedChunks[chunk] = str(reflectedX)
    
    if "\n" in relfectedChunks[len(chunks)-1]:
        relfectedChunks[-1] = relfectedChunks[-1].strip()
        print(relfectedChunks[len(chunks)-1])


    return relfectedChunks

In [10]:
#Vertical Reflection Method Labels
def RV_Coords(chunks):
    relfectedChunks = chunks.copy()
    
    for chunk in range(len(chunks)):
        if(chunk%2 == 0 and chunk != 0):
            reflectedY = 1 - float(chunks[chunk])
            relfectedChunks[chunk] = str(reflectedY)
            
    if "\n" in relfectedChunks[len(chunks)-1]:
        relfectedChunks[-1] = relfectedChunks[-1].strip()
        print(relfectedChunks[len(chunks)-1])

    return relfectedChunks

In [11]:
def transform(t_array, width, height):
    a  = np.array([], np.float32)
    for pair in t_array:
        for v in range(len(pair)):
            if v == 0:
                a = np.append(a, pair[0]/width)
            elif v == 1:
                a = np.append(a, pair[1]/height)
    return np.array([[a[0], a[1]], [a[2], a[3]], [a[4], a[5]], [a[6], a[7]]], np.float32)

In [12]:
def RotationAugmentation(deg):
    for img in glob.glob(inputFolder + "/*.jpg"):
        image  = cv2.imread(img)
        height, width = image.shape[:2] #height is y width is x
        rotation_matrix = cv2.getRotationMatrix2D((width//2, height//2),deg,1)
        rotated_image = cv2.warpAffine(image,rotation_matrix,(width,height))
        labelFile = "./Data/train/labels/{}".format(find_file(img[img.index("Sample")+7:img.index("_jpg")]))
        boxed_image = rotated_image
        try:
            with open(labelFile, "r") as file:
                for line in file:
                    pts = process_points(line, width, height)
                    #print(pts)
                    rotated_pts = rotate((width//2, height//2), pts, np.deg2rad(360-deg))
                    transformed_pts = transform(rotated_pts, width, height)
                    #check_pts = np.array([[(int)((float(transformed_pts[0][0]) * width)),(int)((float(transformed_pts[0][1]) * height))],[(int)((float(transformed_pts[1][0]) * width)), (int)((float(transformed_pts[1][1]) * height))],[(int)((float(transformed_pts[2][0]) * width)), (int)((float(transformed_pts[2][1]) * height))],[(int)((float(transformed_pts[3][0]) * width)), (int)((float(transformed_pts[3][1]) * height))]], np.int32)
                    boxed_image = cv2.polylines(boxed_image, [rotated_pts], True, (0,0,255), 10)
                    write_file(transformed_pts, "./Augmentations/Rotation/{}/labels/".format(deg), img[img.index("Sample"):img.index("_jpg")], "R{}".format(deg), pts[4][0])
        except FileNotFoundError:
            print(f"The file '{labelFile}' was not found.")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
        cv2.imwrite("./Augmentations/Rotation/{}/images/{}_R{}.jpg".format(deg, img[img.index("Sample"):img.index("_jpg")], deg), boxed_image)
        
        

In [13]:
RotationAugmentation(270)

In [14]:
RotationAugmentation(180)

In [15]:
RotationAugmentation(90)

In [16]:
#This is augmentation for reflection horizontal
for img in glob.glob(inputFolder + "/*.jpg"):
    
    file_name = img[img.index("Sample"):img.index("_jpg")]
    labelFile = "./Data/train/labels/{}".format(find_file(file_name))
    reflectionLabel = "./Augmentations/ReflectionHorizontal/labels/"
    image  = cv2.imread(img)
    height,width = image.shape[:2]
    flipped_image = cv2.flip(image, 1)
    boxed_image = flipped_image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                chunks = line.split(' ')
                RH_arr = RH_Coords(chunks)
                boxed_pts = np.array([[(int)((float(RH_arr[1]) * width)),(int)((float(RH_arr[2]) * height))],[(int)((float(RH_arr[3]) * width)), (int)((float(RH_arr[4]) * height))],[(int)((float(RH_arr[5]) * width)), (int)((float(RH_arr[6]) * height))],[(int)((float(RH_arr[7]) * width)), (int)((float(RH_arr[8]) * height))]], np.int32)
                boxed_image = cv2.polylines(boxed_image, [boxed_pts], True, (0,0,255), 10)
                write_file(RH_arr, reflectionLabel, file_name, "RH", chunks[0])
                
    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./Augmentations/ReflectionHorizontal/images/{}_RH.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)


0.8412351081935328
0.6341928033065889
0.752290299051787
0.43131169462679314
0.9166058837831267
0.925115487478726
0.5798687089715536
0.5521723802577194
0.8981108679795771
0.5397520058351568
0.4923413566739606
0.4011670313639679
0.9032336494043277
0.3109632629447498
0.8036316776007498
0.45020149953139643
0.5232739768822243
0.9069639416460226
0.7300320512820513
0.7490784623486045
0.6188824786324786
0.5619294871794872
0.8756998880179171
0.3452467785071724
0.83904
0.8606844152686604
0.8777048383175298
0.8898626306831996
0.3640858254315585
0.7286629775080329
0.3294432287867736
0.8364646464646465
0.8182871572871574
0.5155728715728716
0.8961038961038961
0.2912207792207792
0.4777171717171717
0.7849426063470628
0.7859888936351986
0.9260918253079508
0.6737613245860669
0.7153598346705567
0.5021006564551422
0.5475139800632143
0.822259907610017
0.4908482905982906
0.3836477987421384
0.4035983467055677
0.3610503282275711
0.7330415754923414
0.6990031607099441
0.4906311637080868
0.34159980549477265
0.76

In [40]:
#This is augmentation for reflection vertical
for img in glob.glob(inputFolder + "/*.jpg"):
    file_name = img[img.index("Sample"):img.index("_jpg")]
    labelFile = "./Data/train/labels/{}".format(find_file(file_name))
    reflectionLabel = "./Augmentations/ReflectionVertical/labels/"
    image  = cv2.imread(img)
    height,width = image.shape[:2]
    vert_image = cv2.flip(image, 0)
    boxed_image = vert_image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                chunks = line.split(' ')
                RV_arr = RV_Coords(chunks)
                boxed_pts = np.array([[(int)((float(RV_arr[1]) * width)),(int)((float(RV_arr[2]) * height))],[(int)((float(RV_arr[3]) * width)), (int)((float(RV_arr[4]) * height))],[(int)((float(RV_arr[5]) * width)), (int)((float(RV_arr[6]) * height))],[(int)((float(RV_arr[7]) * width)), (int)((float(RV_arr[8]) * height))]], np.int32)
                boxed_image = cv2.polylines(boxed_image, [boxed_pts], True, (0,0,255), 10)
                write_file(RV_arr, reflectionLabel, file_name, "RV", chunks[0])

    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./Augmentations/ReflectionVertical/images/{}_RV.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)
    

./ForensicSight/train/images\Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6f.jpg
['0', '0.2742927631578947', '0.6839675101530772', '0.6768421052631579', '0.6839675101530772', '0.6768421052631579', '0.27662605435801313', '0.2742927631578947', '0.27662605435801313']
./ForensicSight/train/images\Sample_107B_jpg.rf.b86ec60522c2fc31c4568d83a3707c04.jpg
['0', '0.17434073464912284', '0.8325538894095595', '0.7870956688596491', '0.8325538894095595', '0.7870956688596491', '0.19636832239925017', '0.17434073464912284', '0.19636832239925017']
./ForensicSight/train/images\Sample_120A_jpg.rf.86f20835ecf97d27b62a933befd77db2.jpg
['0', '0.5034722222222222', '0.824945295404814', '0.78125', '0.824945295404814', '0.78125', '0.563578896182835', '0.5034722222222222', '0.563578896182835']
./ForensicSight/train/images\Sample_120B_jpg.rf.e8f0fc1e353b581d12ac6dea5c145bfb.jpg
['0', '0.5473318713450293', '0.7058108436664235', '0.8114035087719298', '0.7058108436664235', '0.8114035087719298', '0.489423778264040

In [184]:
folderPath = "./Augmentations/ReflectionHorizontal/images"
for img in glob.glob(folderPath + "/*.jpg"):
    print(img[img.index("Sample"):img.index("_jpg")])

In [3]:
#Data Validation for Augmentation folders
validate = ["Rotation/90", "Rotation/180", "Rotation/270", "ReflectionHorizontal", "ReflectionVertical"]
for v in validate:
    print("Images: {}: {}".format(v,len(os.listdir("./Augmentations/{}/images/".format(v)))))
    print("Labels: {}: {}".format(v,len(os.listdir("./Augmentations/{}/labels/".format(v)))))
print("{}: {}".format("BoundingBoxed",len(os.listdir("./BoundingBoxed/"))))

Images: Rotation/90: 391
Labels: Rotation/90: 390
Images: Rotation/180: 391
Labels: Rotation/180: 390
Images: Rotation/270: 391
Labels: Rotation/270: 390
Images: ReflectionHorizontal: 391
Labels: ReflectionHorizontal: 390
Images: ReflectionVertical: 391
Labels: ReflectionVertical: 390
BoundingBoxed: 391


In [14]:
#Data Validation for Original Images & Labels
counterImg = 0
inputFolder = './Data/train/images/'
for img in glob.glob(inputFolder + "/*.jpg"):
    counterImg += 1
print('Image:', counterImg)

counterLab = 0
inputFolder = './Data/train/labels/'
for lbl in glob.glob(inputFolder + "/*.txt"):
    counterLab += 1
print('Labels:', counterLab)

Image: 391
Labels: 391


In [162]:
#function to draw a bounding box from points file
for img in glob.glob(inputFolder + "./*.jpg"):
    print(img)
    image  = cv2.imread(img)
    h,w = image.shape[:2]
    d = {}
    labelFile = "./Data/train/labels/{}".format(find_file(img[img.index("Sample")+7:img.index("_jpg")]))
    boxed_image = image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                pts = process_points(line, w, h)
                boxed_image = cv2.polylines(boxed_image, [pts[0:4]], True, (0,0,255), 10)
    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./BoundingBoxed/{}_BB.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)
    

./ForensicSight/train/images/.\Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6f.jpg
./ForensicSight/train/images/.\Sample_107B_jpg.rf.b86ec60522c2fc31c4568d83a3707c04.jpg
./ForensicSight/train/images/.\Sample_120A_jpg.rf.86f20835ecf97d27b62a933befd77db2.jpg
./ForensicSight/train/images/.\Sample_120B_jpg.rf.e8f0fc1e353b581d12ac6dea5c145bfb.jpg
./ForensicSight/train/images/.\Sample_120C_jpg.rf.c8fb9fbcf5a2aba4f5b536682ff13b56.jpg
./ForensicSight/train/images/.\Sample_120D_jpg.rf.d08ca2f76fa2256123a0aec18cb8bb2a.jpg
./ForensicSight/train/images/.\Sample_120E_jpg.rf.6316e18e08908e72df7bf18f2b07d23f.jpg
./ForensicSight/train/images/.\Sample_120F_jpg.rf.4bf9080179ab5800ea617556fa7c1891.jpg
./ForensicSight/train/images/.\Sample_131A_jpg.rf.79b46defcf6dee68ee34e9ae04d27645.jpg
./ForensicSight/train/images/.\Sample_131B_jpg.rf.c3e48b91fbc16944d997b55d243e2561.jpg
./ForensicSight/train/images/.\Sample_132A_jpg.rf.8757d94ed30e4b997a33eb7109315bb9.jpg
./ForensicSight/train/images/.\Sample_132B_

NameError: name 'nvidia' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9a7a2f5b-949c-491a-a647-833490711560' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>